# GPU Computing

<!-- <img src="./imgs/Julia-code-cpu-gpu.png" width=768>

* **host**: CPU + system memory (host memory)
* **device**: GPU with its memory (device memory) -->

### NVIDIA A100 GPU

<img src="./imgs/a100_front.png" width=512px>

**Source:** [NVIDIA whitepaper](https://images.nvidia.com/aem-dam/en-zz/Solutions/data-center/nvidia-ampere-architecture-whitepaper.pdf)

**Streaming Multiprocessor in NVIDIA A100**

<img src="./imgs/a100_SM.png" width=512px>

**Source:** [NVIDIA whitepaper](https://images.nvidia.com/aem-dam/en-zz/Solutions/data-center/nvidia-ampere-architecture-whitepaper.pdf)

| Compute units              | Count            |
|----------------------------|------------------|
| **SMs**                    | 108              |
| **CUDA cores** / FP32 ALUs | 6912 (64 per SM) |
| **Tensor cores**           | 432 (4 per SM)   |

### Topology

<img src="./imgs/gpu_topology.svg" width=500px>

* **host**: CPU + system memory (host memory)
* **device**: GPU with its memory (device memory)
* **SM**: Streaming Multiprocessor

Communication:
* Host-device bandwidth: **31.5 GB/s**
* GPU global memory bandwidth: **1555 GB/s**

## Quick comparison: CPU vs GPU

### AMD EPYC 7763 vs NVIDIA A100

|               | compute units   | maximum clock frequency [GHz] | FP32 peak performance [TFLOPS] |
|:-------------:|:---------------:|:-----------------------------:|:------------------------------:|
| AMD EPYC 7763 |  64 x86 cores   |  3.50                         |  5.0                           |
| NVIDIA A100   | 6912 CUDA cores |  1.41                         | 19.5 (155.9 for Tensor cores)                     |

Most of the [top500](https://top500.org/lists/top500/2023/11/) systems have GPUs as accelerators and they are dominating!

<img src="./imgs/cpu_gpu_fraction.svg" width=768>

**Source:** [J. Apostolakis et al., *Detector simulation challenges for future accelerator experiments.*, Frontiers in Physics 10 (2022)](https://doi.org/10.3389/fphy.2022.913510)

### Differences between CPU and GPU

|                   | CPU                               | GPU                                 |
|:-----------------:|:---------------------------------:|:-----------------------------------:|
| optimized for     | latency and per-core performance  | computational throughput            |
| cores             | complex                           | rather simple                       |
| number of threads | O(100)                            | O(1_000_000)                        |
| thread pinning    | a must for good performance       | not needed at all                   |

### Memory-bound scientific computing

The performance of most scientific codes is **memory-bound** (memory access speed) rather than compute-bound (how fast computations can be done). In a certain time interval, GPUs (and CPUs) can perform more computations than read numbers from memory.

**Peak performance over peak memory bandwidth** (for NVIDIA A100 40GB SXM)

$$
\dfrac{19.5 \ [\textrm{TFLOPS}]}{1.56 \ [\textrm{TB/s}]} \cdot 4 \ \textrm{B} = 50\ \textrm{FLOPs}
$$

To achieve the peak FP32 performance one needs 50 FLOPs per FP32 number (i.e. `Float32`) from memory.

Crucially, the peak memory bandwidth of GPUs is much higher than for CPUs: **~1.56 TB/s** (A100 40GB SXM) vs **~400 GB/s** (2x AMD EPYC 7763).

(→ exercise **saxpy_gpu** and **daxpy_cpu**)

## Julia + GPU (NVIDIA)

Website: https://juliagpu.org/

We'll focus on **NVIDIA GPUs** but there is [support for other GPUs](https://juliagpu.org/) (AMD, Intel, etc.) as well.

The interface to NVIDIA GPU computing is the [CUDA language extension](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html). In Julia there is [CUDA.jl](https://github.com/JuliaGPU/CUDA.jl).

It leverages LLVM, specifically parts of the Julia compiler as well as [GPUCompiler.jl](https://github.com/JuliaGPU/GPUCompiler.jl), to compile **native GPU code**. (compare to `nvcc`)

It provides:

* **High-level abstraction `CuArray`**
* **Tools for writing CUDA kernels**
* **Wrappers to proprietary NVIDIA libraries (e.g. cuBLAS, cuFFT, cuSOLVER, cuSPARSE)**

### Getting CUDA

By default, **it's automatic**. The CUDA toolkit is installed automatically when **using** CUDA.jl for the first time. The only requirement is a working NVIDIA driver.

**Note:** You can readily add CUDA.jl to a Julia environment on a machine without GPUs, say, a login node. See [Precompiling CUDA.jl without CUDA](https://cuda.juliagpu.org/stable/installation/overview/#Precompiling-CUDA.jl-without-CUDA) for more information.

#### System CUDA

You can opt-out of the automatic system by setting a Julia preference, e.g.

```julia
CUDA.set_runtime_version!(v"12.2"; local_toolkit=true)
```

In [ ]:
using CUDA

In [ ]:
CUDA.versioninfo()

In [ ]:
CUDA.functional() # if this works, you're good to go 👍

In [ ]:
device() # the currently selected GPU

## Array programming: `CuArray`

The simplest way to use a GPU is via **vectorized array operations** (e.g. broadcasting). Each of these operations will be backed by one or more GPU kernels, either natively written in Julia or from some application library. As long as your data is large enough you should be able to get nice speedups in many cases.

You use the `CuArray` type, which serves a dual purpose:

* a managed container for GPU memory
* a way to dispatch to operations that execute on the GPU

A `CuArray` is a **CPU object representing GPU memory**.

In [ ]:
x_gpu = CuArray{Float32}(undef, 4)

In [ ]:
CUDA.rand(4) # Note: defaults to Float32

In [ ]:
CUDA.zeros(4)

 We can readily move data to the GPU by converting to `CuArray`.

 <img src="./imgs/cpu_gpu_transfer.svg" width=180px>

In [ ]:
x_cpu = [1,2,3,4]
x_gpu = CuArray(x_cpu) 

(or by using `copyto!` or `copy!` to move it into already allocated memory)

For better performance the data movement between CPU and GPU should be minimized as much as possible.

### Array computations on GPU

In [ ]:
CuArray <: AbstractArray

Therefore, we should be able to do all kind of operations with it, that we'd also do with regular `Array`s. (**duck typing**)

#### Example: Matrix multiplication

In [ ]:
N = 2048
A_gpu = CUDA.rand(N,N);
B_gpu = CUDA.rand(N,N);

In [ ]:
CUDA.@sync A_gpu * B_gpu # we need CUDA.@sync because GPU operations are typically asynchronous

In [ ]:
using BenchmarkTools

@btime CUDA.@sync(A_cpu * B_cpu) setup=(A_cpu = rand(Float32, N,N); B_cpu = rand(Float32, N,N););
@btime CUDA.@sync(A_gpu * B_gpu) setup=(A_gpu = CUDA.rand(N,N); B_gpu = CUDA.rand(N,N););

(Note: `*` for `CuArray`s uses a cuBLAS kernel under the hood)

#### More examples: Broadcasting, `map`, `reduce`, etc.

In [ ]:
CUDA.@sync A_gpu .+ B_gpu; # runs on the GPU!

In [ ]:
CUDA.@sync sqrt.(A_gpu.^2 + B_gpu.^2); # runs on the GPU!

In [ ]:
CUDA.@sync mapreduce(sin, +, A_gpu); # runs on the GPU!

**The power of simple GPU array programming can not be underestimated!** Entire codes (like deep learning frameworks etc.) could be ported to GPU without ever writing a single CUDA kernel manually.

Of course, it isn't always as easy or performance can be improved by writing custom kernels. (→ exercise **heat_diffusion**)

#### "Counter-example:" Scalar indexing

In [ ]:
A_gpu[1]

In [ ]:
CUDA.allowscalar(false)

In [ ]:
A_gpu[1]

**note**:

* The access to an individual element in an array, e.g. scalar indexing, must not be used in array programming.
* You must express arithmetic operations in terms of arrays and treat the `CuArray` array as a whole entity, e.g.

```julia
function gpu_broadcasting!(C, A, B)
    CUDA.@sync C .= A .* B
end
```

#### A few words on memory management

`CuArray`s are managed by Julia's **garbage collector**. If they are unreachable, they will get cleaned up automatically during a GC run. However, keep in mind that the (CPU-focused) GC isn't good at sensing GPU memory pressure.

In [ ]:
CUDA.memory_status()

In [ ]:
x_gpu = CUDA.rand(10_000_000);

In [ ]:
sizeof(x_gpu) |> Base.format_bytes

In [ ]:
CUDA.memory_status()

In [ ]:
x_gpu = nothing; GC.gc(true)

In [ ]:
CUDA.memory_status()

By default CUDA.jl uses a **memory pool** to speed up future allocations. So it might appear as if the objects have not been freed. (On Noctua 2 we have disabled it. You can disable the memory pool with `JULIA_CUDA_MEMORY_POOL=none`.)

We can use `CUDA.unsafe_free!(x_gpu)` and `CUDA.reclaim()` to more aggressively suggest to release the memory.

In [ ]:
x_gpu = CUDA.rand(10_000_000);

In [ ]:
CUDA.memory_status()

In [ ]:
CUDA.unsafe_free!(x_gpu)

In [ ]:
CUDA.memory_status()

Of course, one must be careful with `CUDA.unsafe_free!` because one still has the handle `x_gpu` that now points to free'd memory. But it is fine and very useful in a pattern like this:

```julia
function myfunction(x::CuArray)
    tmp_memory = similar(x)
    expensive_operation!(x, tmp_memory)
    CUDA.unsafe_free!(tmp_memory)
    return x
end
```

In [ ]:
x_gpu = nothing # to be safe :)

## Kernel programming: Writing CUDA kernels

The high-level array programming is most suitable for some types of computations. It may be less efficient, or even not possible for general applications.

**CUDA kernel**: a function that will be executed by all *GPU threads* in parallel.

Based on the index of a thread we can make them operate on different pieces of given data.

(It might be helpful to think of the CUDA kernel as being the body of a loop.)

In [ ]:
function cuda_kernel!(x)
    i = threadIdx().x # the thread index ("loop index")
    x[i] += 1
    return nothing # CUDA kernels should never return anything
end

One can launch the kernel on the GPU with the `@cuda` macro (non-blocking, asynchronous):

In [ ]:
x = CUDA.zeros(1024)

CUDA.@sync @cuda threads=length(x) cuda_kernel!(x)

In [ ]:
x

As you can imaging, kernel programming can become more difficult, especially if you care about performance. A few reasons:

* you need to respect **hardware limitations** of the GPU, e.g. maximum number of GPU threads per block
* **not all operations can readily be expressed as scalar kernels**, e.g. reduction
* kernels execute on the GPU where the **Julia runtime isn't available**

In particular due to the last point, kernel code has limitations
  * no GC
  * no `print` etc. (→ `@cuprint`)
  * code must be fully type inferred (no dynamic dispatch allowed)
  * no `try ... catch ... end`
  * ...

**You can't just write arbitrary Julia code in kernels.** Fortunately though, many things just work and can get you far (see e.g. exercises).

#### Example: Hardware limitation

In [ ]:
x = CUDA.zeros(1025) # one more element than before

CUDA.@sync @cuda threads=length(x) cuda_kernel!(x)

In [ ]:
CUDA.attribute(device(), CUDA.DEVICE_ATTRIBUTE_MAX_THREADS_PER_BLOCK)

### CUDA programming model

<!-- <img src="./imgs/CUDA_programming_model.png" width=1024> -->
<img src="imgs/cuda_prog_model.svg" width=1024>

Conceptual mapping:

* **Grid** of blocks → entire GPU
* **Blocks** of threads → SMs
* **Threads** → CUDA cores

**Note**: up to three dimensions, $(x, y, z)$, can be used to organize the thread blocks and threads in each block.

In [ ]:
function cuda_kernel_blocks!(x)
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x # global thread index
    if i <= length(x) # make sure that we're inbounds (c.f. "loop" iteration range)
        @inbounds x[i] += 1
    end
    return nothing
end

In [ ]:
x = CUDA.zeros(1025);

**execution configuration** for a CUDA kernel:
* `threads`: number of threads in each block
* `blocks`: number of blocks in the grid

In [ ]:
CUDA.@sync @cuda threads=1024 blocks=2 cuda_kernel_blocks!(x)

In [ ]:
x

#### How does our CUDA kernel compare to broadcasting?

In [ ]:
x = CUDA.rand(1024*1024);

function add_one_broadcasting(x)
    CUDA.@sync x .+ 1
end

# same CUDA kernel, but with different execution configurations
function add_one_kernel_1024_1k(x)
    CUDA.@sync @cuda threads=1024 blocks=1024 cuda_kernel_blocks!(x)
end

function add_one_kernel_256_4k(x)
    CUDA.@sync @cuda threads=256 blocks=4*1024 cuda_kernel_blocks!(x)
end

function add_one_kernel_64_16k(x)
    CUDA.@sync @cuda threads=64 blocks=16*1024 cuda_kernel_blocks!(x)
end

@btime add_one_broadcasting(x) setup=(x = CUDA.zeros(1024););
@btime add_one_kernel_1024_1k(x) setup=(x = CUDA.zeros(1024););
@btime add_one_kernel_256_4k(x) setup=(x = CUDA.zeros(1024););
@btime add_one_kernel_64_16k(x) setup=(x = CUDA.zeros(1024););

The performance of a CUDA kernel is affected by the execution configuration (because of the runtime scheduling of thread blocks and threads).

How to obtain a good execution configuration for a CUDA kernel?

### Simplifying kernel launches: [Occupancy API](https://developer.nvidia.com/blog/cuda-pro-tip-occupancy-api-simplifies-launch-configuration/)

Hardcoded execution configuration is not a good idea. A few reasons:

* In reality, the actual maximal number of threads can depend on kernel details, like how many resources the kernel is using.
* You might want to support different GPUs with different hardware limitations.

**Occupancy** measures the ratio of the number of active *warps* per SM to the maximum number of possible warps per SM.

*warp*: a group of 32 parallel threads executing the same instructions.

* Low occupancy usually implies low performance (because of underutilized hardware resources).
* High occupancy, however, may not guarantee the best performance.

The occupancy API is an automatic tool that can be used to obtain *reasonably good* execution configurations.

In [ ]:
kernel = @cuda launch=false cuda_kernel_blocks!(x) # don't launch the kernel

In [ ]:
config = CUDA.launch_configuration(kernel.fun)

Here, the number `blocks` indicates how many blocks we would need to fully occupy the GPU. For a given input `x`, we might need fewer or more blocks.

In [ ]:
threads = min(length(x), config.threads)

In [ ]:
blocks = cld(length(x), threads)

Launching the kernel with the dynamic launch parameters:

In [ ]:
kernel(x; threads, blocks) # calling `kernel` like a regular function

In [ ]:
@btime kernel(x; threads=$threads, blocks=$blocks) setup=(x = CUDA.zeros(1024););

### Introspection

Similar to `@code_*` for CPU there are `@device_code_*` macros for GPU. However, the GPU pendant for `@code_native` is `@device_code_ptx`.

**PTX**: a low-level **p**arallel **t**hread e**x**ecution virtual machine and instruction set architecture used in NVIDIA CUDA programming.

In [ ]:
@device_code_warntype @cuda threads=1024 blocks=1024 cuda_kernel_blocks!(x)

In [ ]:
@device_code_llvm debuginfo=:none @cuda threads=1024 blocks=1024 cuda_kernel_blocks!(x)

In [ ]:
@device_code_ptx @cuda threads=1024 blocks=1024 cuda_kernel_blocks!(x)

## Multi-GPUs in one compute node

**(Note: You can't run this part because you only have a single device in this session.)**

There are **4x NVIDIA A100 GPUs** in a regular Noctua 2 GPU node:

<!-- <img src="./imgs/Noctua2_GPU_node.png" width=320px> -->
<img src="imgs/Noctua2_GPU_node.svg" width=320px>

Since each Julia task gets its own local CUDA execution environment, it is easy to utilize multi-GPUs in one compute node to perform computations in parallel by launching multiple Julia tasks.

In [ ]:
using Base.Threads

In the following we will use `Threads.@spawn` (since multi-threading support is a rather recent addition to CUDA.jl).

In [ ]:
function gpu_computation(A, B, C)
    for i in 1:512
        C = A * B
        A = B * C
        B = C * A
    end
    sin.(B)
    return B
end

function multi_gpu()
    n = 4096
    @sync begin
        # Julia task for the 1st GPU
        @spawn begin
            device!(0) # first GPU
            A = CUDA.rand(n, n)
            B = CUDA.rand(n, n)
            C = CUDA.zeros(n, n)
            println("GPU 1: running gpu_computation")
            CUDA.@sync gpu_computation(A,B,C)
            println("GPU 1: done")
        end
        # Julia task for the 2nd GPU
        @spawn begin
            device!(1) # second GPU
            A = CUDA.rand(n, n)
            B = CUDA.rand(n, n)
            C = CUDA.zeros(n, n)
            println("GPU 2: running gpu_computation")
            CUDA.@sync gpu_computation(A,B,C)
            println("GPU 2: done")
        end
    end
    return nothing
end

In [ ]:
multi_gpu()

In [ ]:
# call CUDA.memory_status() for all GPUs
for dev in devices()
    device!(dev)
    println()
    CUDA.memory_status()
end
device!(0);

## Benchmarking and Profiling

In [ ]:
device!(0)

In [ ]:
A = CUDA.rand(1024, 1024)
B = CUDA.rand(1024, 1024)

@btime CUDA.@sync A .* B;

Note that "allocations" here means CPU allocations. For GPU allocations you can e.g. use `CUDA.@time`.

In [ ]:
CUDA.@time A .* B;

### Integrated profiler: `CUDA.@profile`

In [ ]:
CUDA.@profile A .* B

In [ ]:
CUDA.@profile trace=true A .* B

### External profiler: [NVIDIA Nsight Systems](https://developer.nvidia.com/nsight-systems) (optional)

See https://cuda.juliagpu.org/stable/development/profiling/#External-profilers

**Command**: `CUDA.@profile external=true`

Use [**NVTX.jl**](https://github.com/JuliaGPU/NVTX.jl) to annotate (i.e. label and colorize) code blocks.

<img src="./imgs/nsight_systems.png" width=800px>

## Case study: Three ways to SAXPY on the GPU

**SAXPY** = **S**ingle precision **A** times **X** **P**lus **Y**

→ exercise **saxpy_gpu**

<img src="./imgs/a100_saxpy_results.png" width=1024px>

## Core messages of this notebook

* GPU architecture is optimal for **high throughput computations using millions of threads**.
* For good performance the communication between host and GPU device(s) must be minimized.
* CUDA.jl enables to program NVIDIA GPUs in Julia
    - high-level array abstraction with `CuArray`
        * convenient for some types of computation
        * reasonably good performance is achievable
    - writing custom CUDA kernels
        * flexible for general computations
        * **CUDA programming model**
            - **Grid of blocks** → entire GPU
            - **Blocks of threads** → SMs
            - **Threads** → CUDA cores
        * the performance is influenced by **the execution configuration**, e.g. number of thread blocks and number of threads per block
        * use Occupancy API to obtain an optimal execution configuration for a CUDA kernel
    - using the vendor GPU libraries, e.g. cuBLAS, if possible
* Multi-GPUs in one compute node can be utilized in parallel with multiple Julia tasks.